In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd
from tqdm.auto import tqdm
import time

data = pd.DataFrame(columns=['Gospodarze','Goście', 'Posiadanie piłki H', 'Posiadanie piłki A', 'Sytuacje bramkowe H', 'Sytuacje bramkowe A', 'Strzały na bramkę H', 'Strzały na bramkę A', 'Strzały niecelne H', 'Strzały niecelne A', 'Strzały zablokowane H', 'Strzały zablokowane A', 'Rzuty wolne H', 'Rzuty wolne A', 'Rzuty rożne H', 'Rzuty rożne A', 'Spalone H', 'Spalone A', 'Wrzuty z autu H', 'Wrzuty z autu A', 'Interwencje bramkarzy H', 'Interwencje bramkarzy A', 'Faule H', 'Faule A', 'Czerwone kartki H', 'Czerwone kartki A', 'Żółte kartki H', 'Żółte kartki A', 'Sezon', 'Sędzia', 'Stadion'])

options = Options()
#options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

# Otwieramy stronę z danymi meczu
for iter in tqdm(range(2013,2022,1)):
    driver.get(f"https://www.flashscore.pl/pilka-nozna/polska/pko-bp-ekstraklasa-{iter}-{iter+1}/wyniki/")
    # Czekamy, aż elementy zostaną załadowane
    wait = WebDriverWait(driver, 10)
    while True:
        try:
            more_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "event__more--static")))
            more_button.click()
            time.sleep(5)
        except:
            break

    aaa = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "event__match")))
    pbar = tqdm(total=len(aaa))

    for match in wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "event__match"))):
        match.click()
        wait.until(EC.number_of_windows_to_be(2))
        driver.switch_to.window(driver.window_handles[1])
        stats_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[href='#/szczegoly-meczu/statystyki-meczu']")))
        stats_button.click()
        categories = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "stat__row")))
        team_names = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".participant__participantName")))
        

        row = ['Not found'] * 31
        row[0] = team_names[1].text
        row[1] = team_names[2].text

        for i in categories:
            i = i.text.split('\n')
            match i[1]:
                case 'Posiadanie piłki':
                    row[2] = i[0]
                    row[3] = i[2]
                case 'Sytuacje bramkowe':
                    row[4] = i[0]
                    row[5] = i[2]
                case 'Strzały na bramkę':
                    row[6] = i[0]
                    row[7] = i[2]
                case 'Strzały niecelne':
                    row[8] = i[0]
                    row[9] = i[2]         
                case 'Strzały zablokowane':
                    row[10] = i[0]
                    row[11] = i[2]
                case 'Rzuty wolne':
                    row[12] = i[0]
                    row[13] = i[2]
                case 'Rzuty rożne':
                    row[14] = i[0]
                    row[15] = i[2]  
                case 'Spalone':
                    row[16] = i[0]
                    row[17] = i[2]    
                case 'Wrzuty z autu':
                    row[18] = i[0]
                    row[19] = i[2]   
                case 'Interwencje bramkarzy':
                    row[20] = i[0]
                    row[21] = i[2]   
                case 'Faule':
                    row[22] = i[0]
                    row[23] = i[2]   
                case 'Czerwone kartki':
                    row[24] = i[0]
                    row[25] = i[2]       
                case 'Żółte kartki':
                    row[26] = i[0]
                    row[27] = i[2] 

        row[28] = f'{iter}/{iter+1}'

        stats_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[href='#/szczegoly-meczu/szczegoly-meczu']")))
        stats_button.click()
        driver.refresh()
        
        try:
            referee = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "mi__item__val")))
            row[29] = referee[0].text
            row[30] = referee[1].text
        except:
            pass

        print(row)
        row=pd.DataFrame([row], columns=['Gospodarze','Goście', 'Posiadanie piłki H', 'Posiadanie piłki A', 'Sytuacje bramkowe H', 'Sytuacje bramkowe A', 'Strzały na bramkę H', 'Strzały na bramkę A', 'Strzały niecelne H', 'Strzały niecelne A', 'Strzały zablokowane H', 'Strzały zablokowane A', 'Rzuty wolne H', 'Rzuty wolne A', 'Rzuty rożne H', 'Rzuty rożne A', 'Spalone H', 'Spalone A', 'Wrzuty z autu H', 'Wrzuty z autu A', 'Interwencje bramkarzy H', 'Interwencje bramkarzy A', 'Faule H', 'Faule A', 'Czerwone kartki H', 'Czerwone kartki A', 'Żółte kartki H', 'Żółte kartki A', 'Sezon', 'Sędzia', 'Stadion'])

        #data.loc[len(data)] = row

        data = pd.concat([data,row])
        data.to_excel('output.xlsx', index=False)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        pbar.update(1)




# Zamykamy przeglądarkę
driver.quit()


  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/296 [00:00<?, ?it/s]

['Górnik Zabrze', 'Lechia Gdańsk', '52%', '48%', '22', '14', '7', '6', '8', '6', '7', '2', '22', '17', '6', '5', '4', '2', '27', '21', '4', '7', '14', '21', 'Not found', 'Not found', '3', '1', '2013/2014', 'Gil P. (Pol)', 'Stadion im. Ernesta Pohla (Zabrze)']
['Legia Warszawa', 'Lech Poznań', '49%', '51%', '10', '8', '4', '6', '6', '2', '0', '0', '20', '15', '10', '3', '0', '0', '20', '37', '6', '2', '9', '10', 'Not found', 'Not found', '2', '4', '2013/2014', 'Tojo M. (Jpn)', 'Stadion Wojska Polskiego (Warszawa)']
['Ruch Chorzów', 'Pogoń Szczecin', '45%', '55%', '13', '9', '2', '1', '8', '8', '3', '0', '16', '10', '4', '4', '0', '4', '11', '12', '1', '2', '11', '11', 'Not found', 'Not found', '0', '3', '2013/2014', 'Borski M. (Pol)', 'Stadion Ruchu Chorzów (Chorzów)']
['Wisła Kraków', 'Zawisza Bydgoszcz', '49%', '51%', '11', '19', '6', '5', '4', '8', '1', '6', '18', '26', '5', '4', '3', '0', '13', '16', '4', '4', '26', '18', 'Not found', 'Not found', '0', '5', '2013/2014', 'Zlotek M. (

  0%|          | 0/296 [00:00<?, ?it/s]

['Jagiellonia Białystok', 'Lechia Gdańsk', '48%', '52%', '10', '17', '6', '5', '4', '9', '0', '3', '17', '18', '11', '5', '3', '2', '19', '20', '3', '2', '14', '19', 'Not found', 'Not found', '1', '2', '2014/2015', 'Stefanski D. (Pol)', 'Not found']
['Lech Poznań', 'Wisła Kraków', '53%', '47%', '15', '4', '4', '1', '6', '0', '5', '3', '10', '13', '9', '2', '3', '3', '28', '28', '1', '4', '11', '9', 'Not found', 'Not found', '1', '2', '2014/2015', 'Marciniak S. (Pol)', 'Stadion Miejski (Poznań)']
['Legia Warszawa', 'Górnik Zabrze', '58%', '42%', '12', '6', '2', '2', '8', '2', '2', '2', '15', '17', '5', '2', '4', '1', '28', '19', '2', '1', '10', '14', 'Not found', 'Not found', '0', '1', '2014/2015', 'Frankowski B. (Pol)', 'Stadion Wojska Polskiego (Warszawa)']
['Śląsk Wrocław', 'Pogoń Szczecin', '52%', '48%', '20', '6', '8', '1', '6', '2', '6', '3', '15', '13', '5', '1', '2', '2', '23', '22', '0', '6', '11', '13', 'Not found', 'Not found', '1', '2', '2014/2015', 'Raczkowski P. (Pol)', 'T

...

['Lech Poznań', 'Jagiellonia Białystok', '56%', '44%', '20', '4', '8', '0', '12', '4', 'Not found', 'Not found', 'Not found', 'Not found', '8', '2', '2', '2', 'Not found', 'Not found', '0', '4', '8', '12', 'Not found', 'Not found', 'Not found', 'Not found', '2019/2020', 'Raczkowski P. (Pol)', 'Stadion Miejski (Poznań)']
['Legia Warszawa', 'Pogoń Szczecin', '46%', '54%', '8', '8', '3', '5', '5', '3', 'Not found', 'Not found', 'Not found', 'Not found', '2', '4', '2', '1', 'Not found', 'Not found', '3', '2', '10', '8', 'Not found', 'Not found', '2', '3', '2019/2020', 'Kos D. (Pol)', 'Stadion Wojska Polskiego (Warszawa)']
['Piast Gliwice', 'Cracovia', '51%', '49%', '16', '11', '2', '5', '14', '6', 'Not found', 'Not found', 'Not found', 'Not found', '8', '7', '0', '0', 'Not found', 'Not found', '4', '1', '10', '11', 'Not found', 'Not found', '0', '3', '2019/2020', 'Frankowski B. (Pol)', 'Stadion Miejski w Gliwicach (Gliwice)']
['Śląsk Wrocław', 'Lechia Gdańsk', '56%', '44%', '17', '8', '3',

  0%|          | 0/240 [00:00<?, ?it/s]

['Cracovia', 'Warta Poznań', '46%', '54%', '5', '7', '0', '2', '5', '5', 'Not found', 'Not found', 'Not found', 'Not found', '5', '6', '3', '2', 'Not found', 'Not found', '1', '0', '16', '13', 'Not found', 'Not found', '1', '2', '2020/2021', 'Malec P. (Pol)', 'Stadion MKS Cracovia (Kraków)']
['Jagiellonia Białystok', 'Lechia Gdańsk', '40%', '60%', '15', '18', '7', '6', '8', '12', 'Not found', 'Not found', 'Not found', 'Not found', '6', '8', '3', '4', 'Not found', 'Not found', '5', '5', '14', '12', 'Not found', 'Not found', '5', '4', '2020/2021', 'Jakubik K. (Pol)', 'Stadion Miejski (Białystok)']
['Lech Poznań', 'Górnik Zabrze', '55%', '45%', '15', '15', '8', '7', '7', '8', 'Not found', 'Not found', 'Not found', 'Not found', '10', '6', '1', '1', 'Not found', 'Not found', '6', '7', '13', '7', 'Not found', 'Not found', '2', '1', '2020/2021', 'Aluszyk A. (Pol)', 'Stadion Miejski (Poznań)']
['Legia Warszawa', 'Podbeskidzie B-B', '60%', '40%', '13', '13', '4', '4', '9', '9', 'Not found', 'No

  0%|          | 0/306 [00:00<?, ?it/s]

['Górnik Łęczna', 'Jagiellonia Białystok', '47%', '53%', '10', '14', '2', '4', '5', '5', '3', '5', '16', '18', '4', '5', '0', '2', '14', '11', '3', '2', '20', '12', 'Not found', 'Not found', '4', '3', '2021/2022', 'Gryckiewicz P. (Pol)', 'Stadion GKS Górnik Łęczna (Łęczna)']
['Lech Poznań', 'Zagłębie Lubin', '56%', '44%', '21', '15', '9', '6', '11', '4', '1', '5', '14', '7', '4', '7', '0', '2', '31', '15', '5', '7', '9', '13', 'Not found', 'Not found', '1', '2', '2021/2022', 'Marciniak S. (Pol)', 'Stadion Miejski (Poznań)']
['Legia Warszawa', 'Cracovia', '51%', '49%', '12', '15', '4', '2', '6', '7', '2', '6', '14', '16', '4', '7', '1', '3', '17', '21', '2', '1', '16', '11', 'Not found', 'Not found', '2', '0', '2021/2022', 'Kos D. (Pol)', 'Stadion Wojska Polskiego (Warszawa)']
['Piast Gliwice', 'Radomiak Radom', '51%', '49%', '14', '10', '3', '2', '7', '6', '4', '2', '20', '10', '7', '3', '1', '2', '12', '13', '1', '2', '10', '20', 'Not found', 'Not found', '1', '2', '2021/2022', 'Musia